In [1]:
import gurobipy as grb
from gurobipy import GRB
import networkx as nx


In [2]:
import fio.arith

def layers(G:nx.DiGraph, A: set, B: set, params=None):

    if params is None:
        params = {}

    m = grb.Model()

    for k, v in params.items():
        m.setParam(k, v)

    n = len(G.nodes)

    l = m.addVars(G.nodes, name="l", vtype=GRB.INTEGER)

    r = m.addVars(G.edges, name="r", vtype=GRB.BINARY)

    H = m.addVar(name="H", vtype=GRB.INTEGER)

    for u, v in G.edges:
        m.addConstr(l[v] >= l[u] + 1 - n * r[u, v])
    
    for u in G.nodes:
        m.addConstr(l[u] <= H)
    
    m.setObjectiveN(H, 0, priority=1, name="minimize number of layers")
    m.setObjectiveN(sum(l[v] - l[u] for u, v in G.edges), 1, name="minimize edges length")
    m.setObjectiveN(sum(l[u] for u in A), 2, name="minimize position of elements in A")
    m.setObjectiveN(sum(-l[u] for u in B), 3, name="maximize position of elements in B")
    m.setObjectiveN(sum(r[e] for e in G.edges), 4, priority=1, name="minimize feedback arcs")

    m.setParam(GRB.Param.PoolSolutions, 100)
    

    m.optimize()

    print(sum(r[e].X for e in G.edges))
    

    return {u: fio.arith.float_to_frac(l[u].X) for u in G.nodes}, m.Work



In [3]:
# import random

# def layered_bipartite(l, w, p):
#     G = nx.DiGraph()

#     layers = []

#     for k in range(l):
#         layers.append({(k, i) for i in range(w)})
#         if k > 0:
#             G.add_edges_from((u, v) for u in layers[-2] for v in layers[-1] if random.random() < p)

#     G.add_nodes_from(set().union(*layers))

#     A = set().union(*layers[::2])
#     B = set().union(*layers[1::2])

#     to_remove = {u for u, d in G.degree if d == 0} # pyright: ignore[reportGeneralTypeIssues]

#     G.remove_nodes_from(to_remove)
#     A = A - to_remove
#     B = B - to_remove

#     return G, A, B



In [4]:
# import matplotlib.pyplot as plt
# import numpy as np

# def benchmark_work(
#     sizes,
#     density=0.15,
#     reps=3,
#     seed_base=0,
# ):
#     """
#     sizes: iterable of problem sizes (e.g. [10, 20, 30, ...])
#     density: parameter passed to layered_bipartite
#     reps: number of random instances per size to average over
#     """
#     avg_work = []
#     std_work = []

#     for idx, n in enumerate(sizes):
#         works = []
#         for r in range(reps):
#             # If layered_bipartite supports seeding, you can pass different seeds here
#             G, A, B = layered_bipartite(n, n, density)
#             _, w = layers(G, A, B, params={GRB.Param.OutputFlag: 0})
#             works.append(w)

#         avg = np.mean(works)
#         std = np.std(works)
#         avg_work.append(avg)
#         std_work.append(std)

#         print(f"n={n:4d}  avg work={avg:.2f}  std={std:.2f}")

#     return np.array(sizes), np.array(avg_work), np.array(std_work)

# # Example usage
# sizes = list(range(10, 151, 10))  # 10,20,...,100
# sizes_arr, work_arr, work_std = benchmark_work(sizes, density=0.15, reps=3)

# plt.figure(figsize=(6, 4))
# plt.errorbar(sizes_arr, work_arr, yerr=work_std, fmt="o-", capsize=3)
# plt.xlabel("Problem size n (parameter of layered_bipartite)")
# plt.ylabel("Gurobi work (deterministic units)")
# plt.title("Scaling of MILP work with instance size")
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [5]:
import json

space_age = json.load(open("data/space-age.json"))



In [6]:
G = nx.DiGraph()

for _, r in space_age["recipe"].items():
    if "ingredients" in r and ("category" not in r or r["category"] != "recycling") and ("subgroup" not in r or r["subgroup"] not in ["empty-barrel", "fill-barrel"]) :
        for i in r["ingredients"]:
            G.add_edge( f"item-{i['name']}", f"recipe-{r['name']}")
        for p in r["results"]:
            G.add_edge(f"recipe-{r['name']}", f"item-{p['name']}")

A = set()
B = set()
for u in G.nodes:
    if u.startswith("recipe-"):
        A.add(u)
    
    if u.startswith("item-"):
        B.add(u)

res, work = layers(G, A, B)

for u, l in res.items():
    G.nodes[u]["layer"] = l.numerator

Set parameter Username
Set parameter LicenseID to value 2732003
Academic license - for non-commercial use only - expires 2026-11-03
Set parameter PoolSolutions to value 100
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
PoolSolutions  100

Optimize a model with 1812 rows, 1813 columns and 4815 nonzeros
Model fingerprint: 0xe62c951c
Variable types: 0 continuous, 1813 integer (1191 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 5 objectives (2 combined)...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve...
------------

In [7]:
H = G.copy()

for u, v in G.edges():
    if (G.nodes[v]["layer"] - G.nodes[u]["layer"]) > 1:
        H.remove_edge(u, v)
        dummies = []
        for l in range(G.nodes[u]["layer"] + 1, G.nodes[v]["layer"]):
            dummies.append(f"dummy-{l}-{u}--{v}")
            H.add_node(f"dummy-{l}-{u}--{v}", layer=l)
        nx.add_path(H, [u] + dummies + [v])

    if (G.nodes[u]["layer"] - G.nodes[v]["layer"]) > 1:
        H.remove_edge(u, v)
        dummies = []
        for l in range(G.nodes[v]["layer"] + 1, G.nodes[u]["layer"]):
            dummies.append(f"dummy-{l}-{u}--{v}")
            H.add_node(f"dummy-{l}-{u}--{v}", layer=l)
        nx.add_path(H, [u] + dummies + [v])


len(H.nodes)/len(G.nodes)

4.523349436392914

In [12]:
from itertools import product, combinations, permutations

def layer_ordering(G:nx.DiGraph):
    m = grb.Model()

    H = max(l for u, l in G.nodes(data="layer"))

    layers = [set() for _ in range(H+1)]

    for u, l in G.nodes("layer"):
        layers[l].add(u)

    x = [m.addVars(product(layers[i], repeat=2), vtype=GRB.BINARY) for i in range(H+1)]

    # antisymetry and transitivity constraints
    for k in range(H+1):

        print(len(layers[k]), sum(1 for _ in combinations(layers[k], 2)), sum(1 for _ in permutations(layers[k], 3)))
        # print(len(list(combinations(layers[k], 2))), len(list(permutations(layers[k]))))
        # m.addConstrs(x[k][u, v] + x[k][v, u] <= 1 for u, v in combinations(layers[k], 2))
        # m.addConstrs(x[k][u, v] + x[k][v, w] + x[k][w, u] <= 2 for u, v, w in permutations(layers[k], 3))



layer_ordering(H)
    


8 28 336
29 406 21924
45 990 85140
56 1540 166320
79 3081 474474
78 3003 456456
185 17020 6229320
164 13366 4330584
273 37128 20123376
263 34453 17984466
246 30135 14705880
236 27730 12977640
225 25200 11239200
220 24090 10503240
202 20301 8120400
202 20301 8120400
151 11325 3374850
147 10731 3111990
